### Import Lib

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import CamembertTokenizer, CamembertForSequenceClassification
import locale
import dateparser

### Import CSV

In [32]:
df = pd.read_csv("general_df.csv")

df.head()

,Unnamed: 0.1,Unnamed: 0,user,etoiles,n_avis,localisation,date_avis,titre_avis,text_avis,date_experience,page
0,0,0,Mme Rakotondrasamy Lalanirina,1,3.0,FR,2023-10-20T03:16:24.000Z,Orgueil gestionnaire,"Ça reste une mauvaise expérience, quand ton ge...",Date de l'expérience: 19 octobre 2023,MaBanqueBNPParibas
1,1,1,Florian T,1,2.0,FR,2023-10-19T13:10:44.000Z,Mauvaise expérience en agence physique et en l...,"Mauvaise expérience en agence classique, mon c...",Date de l'expérience: 19 octobre 2023,MaBanqueBNPParibas
2,2,2,Anca Visdei,5,1.0,FR,2023-10-19T15:41:51.000Z,"Chère agence ,","Chère agence ,je dois bous dire que tant mon m...",Date de l'expérience: 19 octobre 2023,MaBanqueBNPParibas
3,3,3,Pat perso,1,5.0,FR,2023-10-18T16:13:38.000Z,Mépris ou incompétence ou les 2,"RV pris à l'agence du Touquet, sur proposition...",Date de l'expérience: 17 octobre 2023,MaBanqueBNPParibas
4,4,4,Sylvia FABRE,2,4.0,FR,2023-10-17T22:22:10.000Z,Pas d'imprimé prévu pour transferts de compte ...,Pour une demande de transfert de fonds d'un co...,Date de l'expérience: 17 octobre 2023,MaBanqueBNPParibas


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170751 entries, 0 to 170750
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0.1     170751 non-null  int64  
 1   Unnamed: 0       170751 non-null  object 
 2   user             170157 non-null  object 
 3   etoiles          170163 non-null  object 
 4   n_avis           169488 non-null  float64
 5   localisation     169487 non-null  object 
 6   date_avis        169488 non-null  object 
 7   titre_avis       169488 non-null  object 
 8   text_avis        169488 non-null  object 
 9   date_experience  168813 non-null  object 
 10  page             168813 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 14.3+ MB


### Cleaning

In [34]:
df_cleaned = df.dropna()

df_cleaned = df_cleaned.drop(columns=["Unnamed: 0", "Unnamed: 0.1","localisation"])

df_cleaned = df_cleaned.rename(columns={"page":"Société"})

df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168806 entries, 0 to 170750
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   user             168806 non-null  object 
 1   etoiles          168806 non-null  object 
 2   n_avis           168806 non-null  float64
 3   date_avis        168806 non-null  object 
 4   titre_avis       168806 non-null  object 
 5   text_avis        168806 non-null  object 
 6   date_experience  168806 non-null  object 
 7   Société          168806 non-null  object 
dtypes: float64(1), object(7)
memory usage: 11.6+ MB


 Transformation étoiles en Int64 de 1 à 5

In [35]:
df_cleaned["etoiles"] = pd.to_numeric(df_cleaned["etoiles"], errors="coerce")

df_cleaned = df_cleaned[df_cleaned["etoiles"].between(1, 5)] # Vérifier que les valeurs soient bien 1 à 5

df_cleaned["etoiles"] = df_cleaned["etoiles"].astype("int64")

df_cleaned.head()
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168806 entries, 0 to 170750
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   user             168806 non-null  object 
 1   etoiles          168806 non-null  int64  
 2   n_avis           168806 non-null  float64
 3   date_avis        168806 non-null  object 
 4   titre_avis       168806 non-null  object 
 5   text_avis        168806 non-null  object 
 6   date_experience  168806 non-null  object 
 7   Société          168806 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 11.6+ MB


Transformation titre et text en STR

In [36]:
df_cleaned["titre_avis"] = df_cleaned["titre_avis"].astype(str)
df_cleaned["text_avis"] = df_cleaned["text_avis"].astype(str)

In [37]:
df_cleaned.head()

,user,etoiles,n_avis,date_avis,titre_avis,text_avis,date_experience,Société
0,Mme Rakotondrasamy Lalanirina,1,3.0,2023-10-20T03:16:24.000Z,Orgueil gestionnaire,"Ça reste une mauvaise expérience, quand ton ge...",Date de l'expérience: 19 octobre 2023,MaBanqueBNPParibas
1,Florian T,1,2.0,2023-10-19T13:10:44.000Z,Mauvaise expérience en agence physique et en l...,"Mauvaise expérience en agence classique, mon c...",Date de l'expérience: 19 octobre 2023,MaBanqueBNPParibas
2,Anca Visdei,5,1.0,2023-10-19T15:41:51.000Z,"Chère agence ,","Chère agence ,je dois bous dire que tant mon m...",Date de l'expérience: 19 octobre 2023,MaBanqueBNPParibas
3,Pat perso,1,5.0,2023-10-18T16:13:38.000Z,Mépris ou incompétence ou les 2,"RV pris à l'agence du Touquet, sur proposition...",Date de l'expérience: 17 octobre 2023,MaBanqueBNPParibas
4,Sylvia FABRE,2,4.0,2023-10-17T22:22:10.000Z,Pas d'imprimé prévu pour transferts de compte ...,Pour une demande de transfert de fonds d'un co...,Date de l'expérience: 17 octobre 2023,MaBanqueBNPParibas


Transformation date_experience en datetime avec un dictionnaire

In [38]:
df_cleaned["date_experience"] = df_cleaned["date_experience"].str.replace("Date de l'expérience:", "").str.strip()

months_fr_to_en = {
    'janvier': 'January',
    'février': 'February',
    'mars': 'March',
    'avril': 'April',
    'mai': 'May',
    'juin': 'June',
    'juillet': 'July',
    'août': 'August',
    'septembre': 'September',
    'octobre': 'October',
    'novembre': 'November',
    'décembre': 'December'
}

def replace_month_fr_to_en(date_str):
    for fr, en in months_fr_to_en.items():
        date_str = date_str.replace(fr, en)
    return date_str

df_cleaned["date_experience"] = df_cleaned["date_experience"].apply(replace_month_fr_to_en)

df_cleaned["date_experience"] = pd.to_datetime(df_cleaned["date_experience"])

print(df_cleaned["date_experience"])



0        2023-10-19
1        2023-10-19
2        2023-10-19
3        2023-10-17
4        2023-10-17
            ...    
170746   2023-10-04
170747   2023-09-28
170748   2023-10-09
170749   2023-10-06
170750   2023-09-26
Name: date_experience, Length: 168806, dtype: datetime64[ns]


In [39]:
df_cleaned.head()

,user,etoiles,n_avis,date_avis,titre_avis,text_avis,date_experience,Société
0,Mme Rakotondrasamy Lalanirina,1,3.0,2023-10-20T03:16:24.000Z,Orgueil gestionnaire,"Ça reste une mauvaise expérience, quand ton ge...",2023-10-19,MaBanqueBNPParibas
1,Florian T,1,2.0,2023-10-19T13:10:44.000Z,Mauvaise expérience en agence physique et en l...,"Mauvaise expérience en agence classique, mon c...",2023-10-19,MaBanqueBNPParibas
2,Anca Visdei,5,1.0,2023-10-19T15:41:51.000Z,"Chère agence ,","Chère agence ,je dois bous dire que tant mon m...",2023-10-19,MaBanqueBNPParibas
3,Pat perso,1,5.0,2023-10-18T16:13:38.000Z,Mépris ou incompétence ou les 2,"RV pris à l'agence du Touquet, sur proposition...",2023-10-17,MaBanqueBNPParibas
4,Sylvia FABRE,2,4.0,2023-10-17T22:22:10.000Z,Pas d'imprimé prévu pour transferts de compte ...,Pour une demande de transfert de fonds d'un co...,2023-10-17,MaBanqueBNPParibas


Transformation date_avis en datetime

In [40]:
df_cleaned["date_avis"] = pd.to_datetime(df_cleaned["date_avis"])



In [41]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168806 entries, 0 to 170750
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   user             168806 non-null  object             
 1   etoiles          168806 non-null  int64              
 2   n_avis           168806 non-null  float64            
 3   date_avis        168806 non-null  datetime64[ns, UTC]
 4   titre_avis       168806 non-null  object             
 5   text_avis        168806 non-null  object             
 6   date_experience  168806 non-null  datetime64[ns]     
 7   Société          168806 non-null  object             
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 11.6+ MB


In [42]:
df_cleaned.head(10)

,user,etoiles,n_avis,date_avis,titre_avis,text_avis,date_experience,Société
0,Mme Rakotondrasamy Lalanirina,1,3.0,2023-10-20 03:16:24+00:00,Orgueil gestionnaire,"Ça reste une mauvaise expérience, quand ton ge...",2023-10-19,MaBanqueBNPParibas
1,Florian T,1,2.0,2023-10-19 13:10:44+00:00,Mauvaise expérience en agence physique et en l...,"Mauvaise expérience en agence classique, mon c...",2023-10-19,MaBanqueBNPParibas
2,Anca Visdei,5,1.0,2023-10-19 15:41:51+00:00,"Chère agence ,","Chère agence ,je dois bous dire que tant mon m...",2023-10-19,MaBanqueBNPParibas
3,Pat perso,1,5.0,2023-10-18 16:13:38+00:00,Mépris ou incompétence ou les 2,"RV pris à l'agence du Touquet, sur proposition...",2023-10-17,MaBanqueBNPParibas
4,Sylvia FABRE,2,4.0,2023-10-17 22:22:10+00:00,Pas d'imprimé prévu pour transferts de compte ...,Pour une demande de transfert de fonds d'un co...,2023-10-17,MaBanqueBNPParibas
5,Véronique,1,5.0,2023-10-17 19:19:25+00:00,Un peu d'amabilité svp,Je me rend à l'agence de Lens pour déposer des...,2023-10-17,MaBanqueBNPParibas
6,Atn Marvaldi,1,1.0,2023-10-16 15:15:54+00:00,Une banque SANS AUCUN RESPECT pour ses…,Une banque SANS AUCUN RESPECT pour ses clients...,2023-10-16,MaBanqueBNPParibas
7,Mon compte Google,1,1.0,2023-10-18 17:08:37+00:00,Une étoile parce que je ne peux pas mettre zer...,"Messagerie ancestrale, pas de conseiller, frai...",2023-10-18,MaBanqueBNPParibas
8,Sandrine Ancian,5,2.0,2023-10-20 14:54:57+00:00,J'ai été en contact téléphonique avec…,J'ai été en contact téléphonique avec une pers...,2023-10-20,MaBanqueBNPParibas
9,Alice Vallegio,1,1.0,2023-10-20 09:58:16+00:00,Alors déjà je laisse une étoile car je…,Alors déjà je laisse une étoile car je n’ai pa...,2023-10-19,MaBanqueBNPParibas


In [43]:


df_cleaned.head()

,user,etoiles,n_avis,date_avis,titre_avis,text_avis,date_experience,Société
0,Mme Rakotondrasamy Lalanirina,1,3.0,2023-10-20 03:16:24+00:00,Orgueil gestionnaire,"Ça reste une mauvaise expérience, quand ton ge...",2023-10-19,MaBanqueBNPParibas
1,Florian T,1,2.0,2023-10-19 13:10:44+00:00,Mauvaise expérience en agence physique et en l...,"Mauvaise expérience en agence classique, mon c...",2023-10-19,MaBanqueBNPParibas
2,Anca Visdei,5,1.0,2023-10-19 15:41:51+00:00,"Chère agence ,","Chère agence ,je dois bous dire que tant mon m...",2023-10-19,MaBanqueBNPParibas
3,Pat perso,1,5.0,2023-10-18 16:13:38+00:00,Mépris ou incompétence ou les 2,"RV pris à l'agence du Touquet, sur proposition...",2023-10-17,MaBanqueBNPParibas
4,Sylvia FABRE,2,4.0,2023-10-17 22:22:10+00:00,Pas d'imprimé prévu pour transferts de compte ...,Pour une demande de transfert de fonds d'un co...,2023-10-17,MaBanqueBNPParibas


In [44]:
df_cleaned.to_csv("df_cleaned.csv", encoding="utf-8", index=False)


In [45]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168806 entries, 0 to 170750
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   user             168806 non-null  object             
 1   etoiles          168806 non-null  int64              
 2   n_avis           168806 non-null  float64            
 3   date_avis        168806 non-null  datetime64[ns, UTC]
 4   titre_avis       168806 non-null  object             
 5   text_avis        168806 non-null  object             
 6   date_experience  168806 non-null  datetime64[ns]     
 7   Société          168806 non-null  object             
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 11.6+ MB
